In [13]:
import numpy as np
import torch
import torchvision
import pandas as pd
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F

In [12]:
# load dataset
dataframe = pd.read_csv("sonar.csv", header=None).replace({"M": 1, "R": 0})
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

XT = torch.from_numpy(X).type(torch.FloatTensor)
YT = torch.from_numpy(Y).type(torch.LongTensor)

In [19]:
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier,self).__init__() 
        self.fc1 = nn.Linear(60,30)
        self.fc2 = nn.Linear(30,5)
        
    #This must be implemented
    def forward(self,x):
        #Output of the first layer
        x = self.fc1(x)
        #Activation function is Relu. Feel free to experiment with this
        x = F.tanh(x)
        #This produces output
        x = self.fc2(x)
        return x
        
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        pred = F.softmax(self.forward(x))
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [20]:
#Initialize the model        
model = MyClassifier()
#Define loss criterion
criterion = nn.CrossEntropyLoss()
#Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [21]:
#Number of epochs
epochs = 10000
#List to store losses
losses = []
for i in range(epochs):
    #Precit the output for Given input
    y_pred = model.forward(XT)
    #Compute Cross entropy loss
    loss = criterion(y_pred,YT)
    #Add loss to the list
    losses.append(loss.item())
    #Clear the previous gradients
    optimizer.zero_grad()
    #Compute gradients
    loss.backward()
    #Adjust weights
    optimizer.step()

c:\users\mohamad\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [22]:
from sklearn.metrics import accuracy_score
print(accuracy_score(model.predict(XT),YT))

c:\users\mohamad\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
c:\users\mohamad\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


1.0


In [23]:
def predict(x):
 #Convert into numpy element to tensor
 x = torch.from_numpy(x).type(torch.FloatTensor)
 #Predict and return ans
 ans = model.predict(x)
 return ans.numpy()